# DATASET
Let's load the dataset and look at what we are working with.
class - basic information about the classes.
students - details about the students
payments - cash payments made by the students in terms of fees,tuition.

In [4]:
#we use this command incase featuretools has not been installed in our system
#!pip install -U featuretools


In [11]:
#we will use this for our data manipulation
import numpy as np
import pandas as pd

#for automated feature engineering
import featuretools as ft



In [15]:
import pandas as pd
#we read our data set
customer=pd.read_csv("C:/Users/RYAN/Desktop/HEARTBEAT/Automated FE/customer.csv",parse_dates=["joined"])

loans=pd.read_csv("C:/Users/RYAN/Desktop/HEARTBEAT/Automated FE/loans.csv",parse_dates=["loan_start","loan_end"])
                     
payments=pd.read_csv("C:/Users/RYAN/Desktop/HEARTBEAT/Automated FE/payments.csv",parse_dates=["payment_date"])

In [10]:
#sample output
customer=pd.read_csv("C:/Users/RYAN/Desktop/HEARTBEAT/Automated FE/customer.csv")
customer.head()

,customer_id,income,joined
0,4769,99467,2012-12-10
1,1961,34097,2011-10-14
2,4566,73407,2016-02-12
3,2566,80373,2016-08-25
4,6705,68166,2013-02-01


In [11]:
loans.head()

,customer_id,loan_amount,loan_end,loan_id,loan_start,loan_type,repaid
0,5564,5316,2015-10-27,10931,2012-05-02,cash,1
1,5564,5894,2010-05-22,11993,2006-08-09,home,0
2,5564,6885,2017-03-08,10669,2012-07-23,cash,0
3,5564,8683,2003-11-30,10852,2000-03-04,other,0
4,5564,6383,2013-09-01,10164,2009-06-10,cash,1


In [12]:
payments.head()

,loan_id,payment_amount,payment_date
0,11028,440,2012-04-01
1,11028,789,2012-04-21
2,11028,468,2012-05-11
3,11028,449,2012-05-27
4,11028,408,2012-06-29


# Manual Feature Engineering Examples.

In [16]:
import numpy as np
#we create month column
customer["joined_month"]=customer["joined"].dt.month

#we create year column
customer["joined_year"]=customer["joined"].dt.year

#we create mean age column
customer["mean_income"]=np.mean(customer["income"])

#we create log age column
customer["log_income"]=np.log(customer["income"])

customer.head()

,customer_id,income,joined,joined_month,joined_year,mean_income,log_income
0,4769,99467,2012-12-10,12,2012,65718.76,11.507581
1,1961,34097,2011-10-14,10,2011,65718.76,10.436965
2,4566,73407,2016-02-12,2,2016,65718.76,11.203775
3,2566,80373,2016-08-25,8,2016,65718.76,11.294434
4,6705,68166,2013-02-01,2,2013,65718.76,11.129701


In [19]:
#groupby by customerid and calculate min,max,mean on loan amount
grp=loans.groupby("customer_id")["loan_amount"].agg(['mean','max','min'])
grp.columns=["mean loan_amount","max loan_amount","min loan_amount"]
grp.head()


,mean loan_amount,max loan_amount,min loan_amount
customer_id,,,
2643,8034.625000,14739,1079
3115,9868.470588,14697,2399
4057,8607.750000,14567,3322
4323,7322.333333,14474,1464
4512,8160.125000,14803,1863


In [21]:
#we now merge with the students dataframe
customer.merge(grp,left_on="customer_id",right_index=True,how="left").head(10)

,customer_id,income,joined,joined_month,joined_year,mean_income,log_income,mean loan_amount,max loan_amount,min loan_amount
0,4769,99467,2012-12-10,12,2012,65718.76,11.507581,NaN,NaN,NaN
1,1961,34097,2011-10-14,10,2011,65718.76,10.436965,NaN,NaN,NaN
2,4566,73407,2016-02-12,2,2016,65718.76,11.203775,NaN,NaN,NaN
3,2566,80373,2016-08-25,8,2016,65718.76,11.294434,NaN,NaN,NaN
4,6705,68166,2013-02-01,2,2013,65718.76,11.129701,NaN,NaN,NaN
5,5205,41928,2011-11-01,11,2011,65718.76,10.643709,NaN,NaN,NaN
6,2918,54074,2011-11-28,11,2011,65718.76,10.898109,NaN,NaN,NaN
7,7831,68269,2017-07-22,7,2017,65718.76,11.131211,NaN,NaN,NaN
8,5400,31421,2011-04-24,4,2011,65718.76,10.355232,NaN,NaN,NaN
9,8286,96269,2014-07-16,7,2014,65718.76,11.474902,NaN,NaN,NaN


# Automated Feature Engineering
# Feature tools

In [22]:
import featuretools as ft
import pandas as pd
customer=pd.read_csv("C:/Users/RYAN/Desktop/HEARTBEAT/Automated FE/customer.csv",parse_dates=["joined"])
loans=pd.read_csv("C:/Users/RYAN/Desktop/HEARTBEAT/Automated FE/loans.csv",parse_dates=["loan_start","loan_end"])
payments=pd.read_csv("C:/Users/RYAN/Desktop/HEARTBEAT/Automated FE/payments.csv",parse_dates=["payment_date"])
#creating entityset
es=ft.EntitySet(id="customer")

#creating an entity from the customer dataframe
es=es.entity_from_dataframe(entity_id="customer",dataframe=customer,
                           index="customer_id",time_index="joined")

#creating an entity from the loans dataframe
es=es.entity_from_dataframe(entity_id="loans",dataframe=loans,
                            variable_types = {'repaid': ft.variable_types.Categorical},
                           index="loan_id",time_index="loan_start"
                           )

#creating an entity from the payments dataframe
es=es.entity_from_dataframe(entity_id="payments",dataframe=payments,
                           make_index=True,index="payment_id",time_index="payment_date")

es

Entityset: customer
  Entities:
    customer [Rows: 25, Columns: 3]
    loans [Rows: 423, Columns: 7]
    payments [Rows: 3497, Columns: 4]
  Relationships:
    No relationships

In [23]:
#All the entities have been successfully added to the entityset.
#Accessing each entity
es["customer"]

Entity: customer
  Variables:
    income (dtype: numeric)
    joined (dtype: datetime_time_index)
    customer_id (dtype: index)
  Shape:
    (Rows: 25, Columns: 3)

In [24]:
es["loans"]

Entity: loans
  Variables:
    customer_id (dtype: numeric)
    loan_amount (dtype: numeric)
    loan_end (dtype: datetime)
    loan_start (dtype: datetime_time_index)
    loan_type (dtype: categorical)
    repaid (dtype: categorical)
    loan_id (dtype: index)
  Shape:
    (Rows: 423, Columns: 7)

In [25]:
es["payments"]

Entity: payments
  Variables:
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    payment_id (dtype: index)
  Shape:
    (Rows: 3497, Columns: 4)

# Relationships


In [27]:
#Relationship between customer and loans
customer_r=ft.Relationship(es["customer"]["customer_id"],
                         es["loans"]["customer_id"])

#Adding the relationship to the entity set
es=es.add_relationship(customer_r)

#Relationship between loans and payments
payments_r=ft.Relationship(es["loans"]["loan_id"],
                         es["payments"]["loan_id"])

#Adding the relationship to the entity set
es=es.add_relationship(payments_r)

es

2018-06-25 06:31:21,770 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: loans.customer_id -> customer.customer_id>


Entityset: customer
  Entities:
    customer [Rows: 25, Columns: 3]
    loans [Rows: 423, Columns: 7]
    payments [Rows: 3497, Columns: 4]
  Relationships:
    loans.customer_id -> customer.customer_id
    payments.loan_id -> loans.loan_id

# Feature Primitives

In [29]:
#aggregations
prmt=ft.list_primitives()
pd.options.display.max_colwidth=150
prmt[prmt["type"]=="aggregation"].head(10)

,name,type,description
0,last,aggregation,Returns the last value.
1,num_unique,aggregation,Returns the number of unique categorical variables.
2,percent_true,aggregation,Finds the percent of 'True' values in a boolean feature.
3,max,aggregation,Finds the maximum non-null value of a numeric feature.
4,std,aggregation,Finds the standard deviation of a numeric feature ignoring null values.
5,sum,aggregation,Counts the number of elements of a numeric or boolean feature.
6,time_since_last,aggregation,Time since last related instance.
7,num_true,aggregation,Finds the number of 'True' values in a boolean.
8,count,aggregation,Counts the number of non null values.
9,min,aggregation,Finds the minimum non-null value of a numeric feature.


In [31]:
#transformations
prmt[prmt["type"]=="transform"].head(10)

,name,type,description
19,absolute,transform,Absolute value of base feature.
20,isin,transform,"For each value of the base feature, checks whether it is in a provided list."
21,second,transform,Transform a Datetime feature into the second.
22,year,transform,Transform a Datetime feature into the year.
23,cum_max,transform,Calculates the max of previous values of an instance for each value in a time-dependent entity.
24,cum_min,transform,Calculates the min of previous values of an instance for each value in a time-dependent entity.
25,days,transform,Transform a Timedelta feature into the number of days.
26,longitude,transform,Returns the second value on the tuple base feature.
27,mod,transform,Creates a transform feature that divides two features.
28,numwords,transform,Returns the words in a given string by counting the spaces.


In [57]:
#creating new features using specified primitives
features, feature_names=ft.dfs(entityset=es, target_entity="customer",
                              agg_primitives=["mean","max","median"],
                              trans_primitives=["years","month","divide"])

pd.DataFrame(features["MONTH(joined)"].head())


,MONTH(joined)
customer_id,
1046,6
1881,7
1961,10
2566,8
2786,8


In [58]:
features.head()

# Deep Feature Synthesis

In [61]:
#Feature with a depth of 1
pd.DataFrame(features["MAX(payments.payment_amount)"].head(10))

In [63]:
#Feature with a depth of 2
pd.DataFrame(features['MONTH(customer.MEAN(payments.payment_amount))'].head(10))

# Automated Feature Synthesis

In [ ]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='customer', 
                                 max_depth = 2)

features.head()